In [ ]:
%pylab inline
import csv
import imageio
import numpy as np
import os
from skimage.metrics import structural_similarity

In [ ]:
def calculate_psnr(img1, img2, max_value=255):
    """"Calculating peak signal-to-noise ratio (PSNR) between two images."""
    mse = np.mean((np.array(img1, dtype=np.float32) - np.array(img2, dtype=np.float32)) ** 2)
    if mse == 0:
        return 100
    return 20 * np.log10(max_value / (np.sqrt(mse)))

# calculate_psnr(gt_img, ours_img)

In [ ]:
def calculate_ssim(img1, img2):
    return structural_similarity(img1, img2, multichannel=True)

In [ ]:
def max_shifted_metric(function, img1, img2, max_shift=1):
    max_value = 0
    
    for y_shift in range(-max_shift, max_shift + 1):
        y_start = max(0, y_shift)
        y_end = min(img1.shape[0], img1.shape[0] + y_shift)
        for x_shift in range(-max_shift, max_shift + 1):
            x_start = max(0, x_shift)
            x_end = min(img1.shape[1], img1.shape[1] + x_shift)
        
            img1_shifted = img1[y_start:y_end, x_start:x_end]
            img2_cropped = img2[:img1_shifted.shape[0], :img1_shifted.shape[1]]
            
            value = function(img1_shifted, img2_cropped)
#             print(y_shift, x_shift, value)
            if value > max_value:
                max_value = value

    return max_value

# max_shifted_metric(calculate_psnr, gt_img, ours_img)

In [ ]:
gt_path = r"G:\OmniPhotos\Data\GT-Replica\{dataset}\cubmap_image\image\{dataset}_{index:04}_{side}.jpg"
output_path = r"G:\\OmniPhotos\progress\results\GT quantitative comparison"
comparison = "Our method (GT inputs)"
ours_path = os.path.join(output_path, comparison, "{dataset}-Replica-cubemaps-{index:04}_{side}.png")

datasets = ['apartment_0', 'hotel_0', 'office_0', 'office_4', 'room_0', 'room_1']
cubemap_sides = 'FLRBUD'

## Compute PSNR

In [ ]:
total_count = 0
total_psnr = 0
total_psnr_squared = 0

with open(os.path.join(output_path, comparison +" - PSNR.csv"), 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_NONE)
    csvwriter.writerow(["Dataset", "Index", "Face", "PSNR"])

    for dataset in datasets:
        dataset_count = 0
        dataset_psnr = 0

        print(dataset, end='')
        for index in range(81):
            print('.', end='')
            for side in cubemap_sides:
                ## Paths to images
                gt_image_path = gt_path.format(**locals())
                ours_image_path = ours_path.format(**locals())
                
                ## Load images
                gt_img = imageio.imread(gt_image_path)
                ours_img = imageio.imread(ours_image_path)

                ## Compute metrics
                psnr = max_shifted_metric(calculate_psnr, gt_img, ours_img)
                
                ## Log result
#                 print(f"{dataset} -- {index:02} -- {side} -- PSNR: {psnr:.2f}")
                csvwriter.writerow([dataset, index, side, psnr])
                dataset_count += 1
                dataset_psnr += psnr
                total_psnr_squared += psnr * psnr
                break
            break

        psnr = dataset_psnr / dataset_count
        print(f'PSNR: {psnr:.2f}')

        total_count += dataset_count
        total_psnr += dataset_psnr
        break

mean_psnr = total_psnr / total_count
stdev_psnr = sqrt(total_psnr_squared / total_count - pow(mean_psnr, 2))
sem_psnr = stdev_psnr / sqrt(total_count)
print(f"PSNR: {mean_psnr:.2f} +/- {sem_psnr:.2f}")

In [ ]:
subplot(1, 2, 1); imshow(gt_img)
subplot(1, 2, 2); imshow(ours_img)

## Compute SSIM

In [ ]:
total_count = 0
total_ssim = 0
total_ssim_squared = 0

with open(os.path.join(output_path, comparison +" - SSIM.csv"), 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_NONE)
    csvwriter.writerow(["Dataset", "Index", "Face", "SSIM"])

    for dataset in datasets:
        dataset_count = 0
        dataset_ssim = 0

        print(dataset, end='')
        for index in range(81):
            print('.', end='')
            for side in cubemap_sides:
                ## Paths to images
                gt_image_path = gt_path.format(**locals())
                ours_image_path = ours_path.format(**locals())
                
                ## Load images
                gt_img = imageio.imread(gt_image_path)
                ours_img = imageio.imread(ours_image_path)

                ## Compute metrics
                ssim = max_shifted_metric(calculate_ssim, gt_img, ours_img)
                
                ## Log result
                csvwriter.writerow([dataset, index, side, ssim])
                dataset_count += 1
                dataset_ssim += ssim
                total_ssim_squared += ssim * ssim
            break

        ssim = dataset_ssim / dataset_count
        print(f'SSIM: {ssim:.4f}')

        total_count += dataset_count
        total_ssim += dataset_ssim
        break

mean_ssim = total_ssim / total_count
stdev_ssim = sqrt(total_ssim_squared / total_count - pow(mean_ssim, 2))
sem_ssim = stdev_ssim / sqrt(total_count)
print(f"SSIM: {mean_ssim:.4f} +/- {sem_ssim:.4f}")

## Compute statistics from CSVs

In [ ]:
computed_dir = os.path.join(output_path, "computed")

for csv_filename in os.listdir(computed_dir):
    total_count = 0
    total_value = 0
    total_value_squared = 0
    
    if csv_filename[-4:] != '.csv':
        continue
    
    value_per_dataset = {}

    with open(os.path.join(computed_dir, csv_filename)) as csvfile:
        csvreader = csv.reader(csvfile, delimiter=',', quoting=csv.QUOTE_NONE)
        next(csvreader)  # skip header
        for row in csvreader:
            dataset, index, side, value = tuple(row)
            index = int(index)
            value = float(value)

            ## Skip images on the camera circle
            if index in [0,1,7,8,16,35,45,64,72,73,79,80]:
                continue
            
            ## We're ignoring this dataset as the scene is way too close to handle.
            if dataset in ['office_4']:
                continue

            ## We're focusing on the views around the equator, as that's where people mostly look
            if side in ['D', 'U']:
                continue

            value_per_dataset[dataset] = value_per_dataset.get(dataset, 0) + value

            total_count += 1
            total_value += value
            total_value_squared += value * value

    mean = total_value / total_count
    stdev = sqrt(total_value_squared / total_count - pow(mean, 2))
    sem = stdev / sqrt(total_count)

    if "PSNR" in csv_filename:
        # round to 2 significant digits
        mean = np.round(mean, 2)
        sem  = np.round(sem,  2)
        print(f"{csv_filename} : {mean:.2f} +/- {sem:.2f}")
    else:
        # round to 3 significant digits
        mean = np.round(mean, 3)
        sem  = np.round(sem,  3)
        print(f"{csv_filename} : {mean:.3f} +/- {sem:.3f}")
#     for dataset, value in value_per_dataset.items():
#         print(f"  - {dataset}: {value/(total_count/len(value_per_dataset)):.4}")

----
## Format stats ready for LaTeX table

In [ ]:
# ## all found files
# all_files = os.listdir(output_path)
# csv_files = [e for e in all_files if e[-8:] == 'SSIM.csv']
# row_names = [e.split(' - ')[0] for e in csv_files]

## rows in the same order as in Table 1 (for easier updating)
row_names = [
    'MegaParallax-cylinder-3m',
    'MegaParallax-plane-3m',
    'Parallax360-cylinder-3m',
    #
    'Our complete method',
    'Our method (GT inputs)',
    #
    'No robust data term',
    'No normalised residuals',
    'Huber-depth-sres',
    'Optimising depth (not inverse)',
    'DIS flow',
    'No flow (linear blending)',
    #
    'Low-res proxy',
    'High-res proxy',
    #
    'Less smoothness',
    'More smoothness', 
    #
    'Fewer images (45-DIS)',
    'Fewer images (30-DIS)',
    'Fewer images (15-DIS)',
]

for row_name in row_names:

    row_text = row_name.ljust(35) + "&  ...  "
    
    for metric in ['LPIPS', 'SSIM', 'PSNR']:
        csv_filename = f"{row_name} - {metric}.csv"
        
        total_count = 0
        total_value = 0
        total_value_squared = 0

        with open(os.path.join(output_path, csv_filename)) as csvfile:
            csvreader = csv.reader(csvfile, delimiter=',', quoting=csv.QUOTE_NONE)
            next(csvreader)  # skip header
            for row in csvreader:
                dataset, index, side, value = tuple(row)
                index = int(index)
                value = float(value)

                ## Skip images on the camera circle
                if index in [0,1,7,8,16,35,45,64,72,73,79,80]:
                    continue

                ## We're ignoring this dataset as the scene is way too close to handle.
                if dataset in ['office_4']:
                    continue

                ## We're focusing on the views around the equator, as that's where people mostly look
                if side in ['D', 'U']:
                    continue

                total_count += 1
                total_value += value
                total_value_squared += value * value

        mean = total_value / total_count
        stdev = sqrt(total_value_squared / total_count - pow(mean, 2))
        sem = stdev / sqrt(total_count)

        if "PSNR" in csv_filename:
            # round to 2 significant digits
            mean = np.round(mean, 2)
            sem  = np.round(sem,  2)
            row_text += f"&  {mean:.2f}$\\pm${sem:.2f}  "
        else:
            # round to 3 significant digits
            mean = np.round(mean, 3)
            sem  = np.round(sem,  3)
            row_text += f"&  {mean:.3f}$\\pm${sem:.3f}  "

    row_text += "\\\\"
    print(row_text)